<a href="https://colab.research.google.com/github/eth0-02/Astro-Theme-Creek/blob/master/unpivotor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
import pandas as pd, re

print("📂 Please upload one or more CSV files…")
uploaded = files.upload()

MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
MONTH_TO_NUM = {m:i for i,m in enumerate(MONTHS,1)}

def find_et_month_cols(df):
    et_cols = []
    for c in df.columns:
        lc = c.lower()
        if lc.startswith("et") and "_mm_" in lc:
            part = lc.split("_mm_")[-1][:3].capitalize()
            if part in MONTHS:
                et_cols.append(c)
    return et_cols

def extract_month_name(c):
    return re.split(r"(?i)_mm_", c)[-1][:3].capitalize()

def transform_clean(df):
    df = df.copy()
    et_cols = find_et_month_cols(df)
    if not et_cols:
        return None

    # keep fid if available
    keep_cols = []
    if "fid" in df.columns:
        keep_cols.append("fid")

    # main identifier column
    id_col = None
    for cand in ["county","BNAME","HYBAS_ID","basin"]:
        if cand in df.columns:
            id_col = cand
            break
    if not id_col:
        id_col = df.columns[0]  # fallback to first col
    keep_cols.append(id_col)

    # melt
    long_df = df.melt(id_vars=keep_cols, value_vars=et_cols,
                      var_name="Month_raw", value_name="ET_mm")

    long_df["Month"] = long_df["Month_raw"].apply(extract_month_name)
    long_df["Month_Number"] = long_df["Month"].map(MONTH_TO_NUM)
    long_df["ET_mm"] = pd.to_numeric(long_df["ET_mm"], errors="coerce").fillna(0)

    return long_df[keep_cols + ["Month","Month_Number","ET_mm"]]

for fname in uploaded.keys():
    df = pd.read_csv(fname)
    long_df = transform_clean(df)
    if long_df is None:
        print(f"⚠️ Skipped {fname} (no ET month columns found)")
        continue

    outname = re.sub(r"\.csv$","",fname)+"_LONG.csv"
    long_df.to_csv(outname,index=False)
    print(f"✅ Saved {outname} with columns: {list(long_df.columns)}")
    files.download(outname)


📂 Please upload one or more CSV files…


Saving BLUE ET SUB.csv to BLUE ET SUB (6).csv
Saving ET SUB.csv to ET SUB (7).csv
Saving GREEN ET SUB.csv to GREEN ET SUB (6).csv
✅ Saved BLUE ET SUB (6)_LONG.csv with columns: ['fid', 'HYBAS_ID', 'Month', 'Month_Number', 'ET_mm']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved ET SUB (7)_LONG.csv with columns: ['fid', 'HYBAS_ID', 'Month', 'Month_Number', 'ET_mm']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved GREEN ET SUB (6)_LONG.csv with columns: ['fid', 'HYBAS_ID', 'Month', 'Month_Number', 'ET_mm']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>